# Overview



In [21]:
#Core imports

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
#Import data

data = pd.read_csv('../ads/Advertisement_Transcripts.csv')

In [3]:
#explore data
data.info()

#5 columns with 1,9935 enties 
#the label column is blank and will be dropped for exploration 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936 entries, 0 to 1935
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Category         1935 non-null   object 
 1   Advertiser       1935 non-null   object 
 2   Product_or_spot  1935 non-null   object 
 3   Ad_copy          1935 non-null   object 
 4   label            0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 75.8+ KB


In [4]:
if 'label' in data.columns:
    data.drop(columns='label', inplace= True)

data.head()

,Category,Advertiser,Product_or_spot,Ad_copy
0,Advocacy,23 Women,23 Women,Women make up seventy percent of the labor for...
1,Advocacy,AARP,AARP,It's right in your own backyard.\nWhile it may...
2,Advocacy,Above the Influence,Above the Influence (PSA),"I thought I respected myself. That is, until I..."
3,Advocacy,Adopt-A-Horse,Adopt-A-Horse,If there's a place in your heart for a wild mu...
4,Advocacy,Amberalert.org,Amberalert.org,"When someone abducts a child, they're not abou..."


In [7]:
#Checking for blank rows 

# show blank rows
blank_df = data.isnull().any(axis=1)
blank_rows = data[blank_df]
print(blank_rows)

# drop row 1935 
if 1935 in data.index:
    if data.loc[1935].isnull().all():
        data.drop(index=1935, inplace=True)

print(data.isnull().sum())

Empty DataFrame
Columns: [Category, Advertiser, Product_or_spot, Ad_copy]
Index: []
Category           0
Advertiser         0
Product_or_spot    0
Ad_copy            0
dtype: int64


In [20]:
#Explore groups
category_counts = data['Category'].value_counts()


# print the results
print('Category counts:\n', category_counts)


Category counts:
 Automotive               289
Food & Beverage          244
Home & Personal Care     215
Travel                   193
Advocacy                 156
Financial                142
Beauty                   103
Services                  98
Media & Entertainment     74
Health Care               73
Alcohol & Spirits         64
Restaurants               54
Retail                    47
Telecommunications        44
Insurance                 43
Education                 24
Electronics               16
Technology                13
Sports                    12
Residential               11
Political                  8
Agricultural               5
Appliances                 5
HeaLth Care                2
Name: Category, dtype: int64


AttributeError: 'list' object has no attribute 'sum'

In [17]:
data[].value_counts()

SyntaxError: invalid syntax (2700824956.py, line 1)